# Modifying Rates

Sometimes we want to change the nuclei involved in rates to simplify our network.  Currently,
pynucastro supports changing the products.  Here's an example.

In [1]:
import pynucastro as pyna

ModuleNotFoundError: No module named 'microphysics'

In [2]:
reaclib_library = pyna.ReacLibLibrary()

NameError: name 'pyna' is not defined

We want to model ${}^{12}\mathrm{C} + {}^{12}\mathrm{C}$ reactions.  There are 3 rates involved.

In [3]:
filter = pyna.RateFilter(reactants=["c12", "c12"])
mylib = reaclib_library.filter(filter)
mylib                       

NameError: name 'pyna' is not defined

The rate ${}^{12}\mathrm{C}({}^{12}\mathrm{C},n){}^{23}\mathrm{Mg}$ is quickly followed by ${}^{23}\mathrm{Mg}(n,\gamma){}^{24}\mathrm{Mg}$, so we want to modify that rate sequence to just be ${}^{12}\mathrm{C}({}^{12}\mathrm{C},\gamma){}^{24}\mathrm{Mg}$

In [4]:
r = mylib.get_rate("c12 + c12 --> n + mg23 <cf88_reaclib__reverse>")
r

NameError: name 'mylib' is not defined

This has the Q value:

In [5]:
r.Q

NameError: name 'r' is not defined

Now we modify it

In [6]:
r.modify_products("mg24")
r

NameError: name 'r' is not defined

and we see that the Q value has been updated to reflect the new endpoint

In [7]:
r.Q

NameError: name 'r' is not defined

Now let's build a network that includes the nuclei involved in our carbon burning.  We'll start by leaving off the ${}^{23}\mathrm{Mg}$

In [8]:
mylib2 = reaclib_library.linking_nuclei(["p", "he4", "c12", "o16", "ne20", "na23", "mg24"])

NameError: name 'reaclib_library' is not defined

Now we add in our modified rate

In [9]:
mylib2 += pyna.Library(rates=[r])

NameError: name 'mylib2' is not defined

In [10]:
mylib2

NameError: name 'mylib2' is not defined

In [11]:
rc = pyna.RateCollection(libraries=[mylib2])
fig = rc.plot(rotated=True, curved_edges=True, hide_xalpha=True)

NameError: name 'pyna' is not defined